# Imitation learning training script!!!!

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim

import os
import pandas as pd
import numpy as np
import pathlib

import cv2

## Check if CUDA is available

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Data loader

In [ ]:
def load_data_session(session_path):
    
    res = []
    
    session_path = pathlib.Path(session_path)
    labels_path = session_path / "labels.csv"
    
    labels_df = pd.read_csv(labels_path, header=None ,skiprows=1)
    
    for index, row in labels_df.iterrows():
        
        img_path = session_path / row[0]
        img = cv2.imread(str(img_path))
        
        if img is None:
            print("cannot load image:", img_path)
        
        res.append((img, row[1], row[2]))
        
    npres = np.array(res, dtype=object)
    np.random.shuffle(npres)
    
    return npres

In [44]:
res = load_data_session("/home/kazuh/Documents/UBC/ENPH_353/training_data/session_20251126_173028_839imgs")

In [46]:
def load_all_sessions(base_path):
    all_data = []
    
    base_path = pathlib.Path(base_path)
    
    for session_dir in base_path.iterdir():
        if session_dir.is_dir():
            session_data = load_data_session(session_dir)
            all_data.append(session_data)
    
    all_data_np = np.concatenate(all_data, axis=0)
    np.random.shuffle(all_data_np)
    
    return all_data_np

In [47]:
all_data = load_all_sessions("/home/kazuh/Documents/UBC/ENPH_353/training_data/all_sessions")

In [50]:
train_ratio = 0.8
num_samples = all_data.shape[0]
num_train = int(num_samples * train_ratio)

train_data = all_data[:num_train]
val_data = all_data[num_train:]

## Design Model

In [4]:
class JackDrivingModel(nn.Module):

    def __init__(self, img_channels = 3, img_h = 600, img_w = 800, act_dim = 2):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(img_channels, 24, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU()
        )

        with torch.no_grad():
            dummy = torch.zeros(1, img_channels, img_h, img_w)
            conv_out = self.conv(dummy)
            conv_feat_dim = conv_out.view(1, -1).shape[1]

        self.fc = nn.Sequential(
            nn.Linear(conv_feat_dim, 100), nn.ReLU(),
            nn.Linear(100, 50), nn.ReLU(),
            nn.Linear(50, act_dim)  # [vx, wz]
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

        